# Soustraction d'image moyenne sur une vidéo

## Etape 1 : Choix de la vidéo

In [1]:
import ipywidgets as widgets
import os


allfiles=os.listdir(os.getcwd()+"/Vidéos")
listeVideo=[filename for filename in allfiles if  filename[-4:] in [".mp4",".avi",".mov",".3gp"]]

nomVideo = widgets.Dropdown(options=listeVideo,
    description='Choisissez une vidéo :',
    disabled=False,
)
display(nomVideo)

out = widgets.Output()


def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        out.clear_output()
        with out :
            print("Vous avez choisi %s" % change['new'])
nomVideo.observe(on_change)
display(out)

Dropdown(description='Choisissez une vidéo :', options=('BestBilliardShots.mp4', 'BestTableTennisJune2019.mp4'…

Output()

In [7]:
file=nomVideo.value
print(file)

# lecture de la vidéo

from IPython.display import HTML

HTML("""
<video width="320" height="240" controls>
  <source src="Vidéos/file" type="video/mp4">
</video>
""")


echanges_1(1).mov


## Etape 2 : Choix des paramètres de travail

In [9]:
startTime = input("Début de l'extrait (hh:mm:ss) : ")
if (startTime=='d'):
    startSecond=0
else:
    temp=startTime.split(':')
    startSecond = int(temp[0])*3600+int(temp[1])*60+int(temp[2])
    
stopTime = input("Fin de l'extrait (hh:mm:ss) : ")
if (stopTime=='f'):
    stopSecond=-1
else:
    temp=stopTime.split(':')
    stopSecond = int(temp[0])*3600+int(temp[1])*60+int(temp[2])

if (stopSecond!=-1):
    if (startSecond>stopSecond):
        print("Time codes erronés")
        
#saveFPS = int(input("Nombre d'images par seconde à sélectionner : "))
tailleIntervalle = int(input("Nombre d'images pour calculer l'image moyenne : "))
posIm = int(input("Position de l'image dans cet intervalle : "))-1


if (posIm>=tailleIntervalle) or (posIm<0):
    print("Position de l'image erronée")


Début de l'extrait (hh:mm:ss) : d
Fin de l'extrait (hh:mm:ss) : f
Nombre d'images par seconde à sélectionner : 30
Nombre d'images pour calculer l'image moyenne : 5
Position de l'image dans cet intervalle : 2


## Etape 3 : Soustraction d'image moyenne

In [10]:
import cv2,numpy


def video2listImage(adresseVideo,staSec,stoSec): # Renvoie une liste d'image sous forme d'array
    video = cv2.VideoCapture(adresseVideo)
    videoFPS = video.get(cv2.CAP_PROP_FPS)
    print(videoFPS)
    
    # Première étape, aller au time code de départ
    continuer=True
    compteur=0
    while continuer:
        isRead = video.read()
        if not isRead:
            continuer = False
            print("Time code de départ trop grand")
            return None
        else:
            compteur+=1
            if (compteur/videoFPS>staSec):
                continuer = False
    
    compteur-=1
    if staSec!=0:
        print("Time code de début atteint",compteur/videoFPS)
    
    imlist=[]
    continuer = True
    
    while continuer:
        isRead , frame = video.read()  # A-t-on lu une image et si oui cette image
        if not isRead:
            continuer = False
        else:
            #frameTimeCode = compteur/videoFPS
            imlist.append(frame)
            compteur+=1
            
            if (stoSec!=-1): #Si on ne va pas jusqu'à la fin de la vidéo
                if(compteur/videoFPS>stoSec):
                    print("Time code de fin atteint",compteur/videoFPS)
                    continuer=False
    
    if (compteur/videoFPS<stoSec):
        print("Time code de fin trop grand")
        return None
    
    return imlist,videoFPS

def subImageMoyArr(imListArr,PosIm): # Renvoie un tableau OpenCV
    n=len(imListArr)
    imCentre=imListArr[PosIm]
    imMoy=calcImageMoyenneArr(imListArr)
    imSub = cv2.subtract(imCentre,imMoy)
    return imSub

def calcImageMoyenneArr(imListArr): # Renvoie array
    w,h=len(imListArr[0]), len(imListArr[0][0])
    arr=numpy.zeros((h,w,4),numpy.float)
    images = numpy.array([image for image in imListArr])
    # Round values in array and cast as 8-bit integer
    arr=numpy.array(numpy.mean(images,axis=(0)),dtype=numpy.uint8)
    return arr

def SubImageMoySurListe(imList,tailleMoy,PosIm):
    imlistMoy = [0]*tailleMoy
    imlistSubMoy = [0]*(len(imList)-tailleMoy+1)
    for i in range(PosIm,len(imList)-(tailleMoy-PosIm)):
        # On selectionne les images autour de celle traitée
        for j in range(tailleMoy):
            imlistMoy[j]=imList[j+i-PosIm]
        imlistSubMoy[i-PosIm]=subImageMoyArr(imlistMoy,PosIm)
    return imlistSubMoy



imList, FPS = video2listImage("Vidéos/"+file,startSecond,stopSecond) # Tableau d'array
if (imList!=None):
    imListSubMoy=SubImageMoySurListe(imList, tailleIntervalle,posIm) # Les mêmes images, auxquelles on a enlevé les images moyennes
    print('OK')
print("Done")


29.984649496844618
30 29.984649496844618 29.984649496844618
OK
Done


## Etape 4 : Résultat

In [11]:
import time
import cv2
"""
def readVideo(nomFichier):
    cap = cv2.VideoCapture(nomFichier)
    videoFPS = cap.get(cv2.CAP_PROP_FPS)
    # Check if camera opened successfully
    if (cap.isOpened()== False): 
        print("Error opening video  file")
    # Read until video is completed
    while(cap.isOpened()):
      # Capture frame-by-frame
      ret, frame = cap.read()
      if ret == True:
        # Display the resulting frame
        cv2.imshow('Frame', frame)
        time.sleep(1/videoFPS)
        # Press Q on keyboard to  exit
        if cv2.waitKey(25) & 0xFF == ord('q'):
          break
      # Break the loop
      else: 
        break
    # When everything done, release 
    # the video capture object
    cap.release()"""

#Lecture de vidéo
def listImage2Video(imList,writeFPS,nomFichier):
    height,width,layers=imList[0].shape
    # choose codec according to format needed
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    video=cv2.VideoWriter(nomFichier, fourcc, writeFPS,(width,height))
    for img in imList:
        video.write(img)
    return video
   
videoWriteFile="VidéosSUB/SUB "+file
videoSub = listImage2Video(imListSubMoy,FPS, videoWriteFile)
videoSub.release()
#readVideo(videoWriteFile)

from IPython.display import HTML

HTML("""
<video width="320" height="240" controls>
  <source src="videoWriteFile" type="video/mp4">
</video>
""")


In [5]:
## Effacer les fichiers générés
import os
allfiles=os.listdir(os.getcwd()+"/VidéosSUB")
for file in allfiles:
    os.remove("VidéosSUB/"+file)